In [ ]:
# libraries

import seaborn as sns
import xarray as xr
import numpy as np
import pandas as pd
import intake
import dask
import regionmask
import warnings
from pathlib import Path
from tqdm import tqdm
from xmip.preprocessing import combined_preprocessing, replace_x_y_nominal_lat_lon
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# data

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
    col = intake.open_esm_datastore(url)

In [ ]:
# load 6-hourly data with meridional and zonal wind, air temperature, surface pressure, and spefific humidity

query = dict(source_id=['GFDL-CM4'],# , 'MRI-ESM2-0', 'CESM2', 'CanESM5'],
             table_id='6hrLev',
             experiment_id=['amip', 'ssp585'],
             variable_id=['va', 'ua', 'ta', 'ps', 'hus']
)

cat = col.search(**query)

In [ ]:
# load data into dictionary
# keys are constructed as 'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

warnings.filterwarnings("ignore")

z_kwargs = {'consolidated': True, 'decode_times':True}

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dset_dict = cat.to_dataset_dict(zarr_kwargs=z_kwargs, preprocess=combined_preprocessing)

dset_dict.keys()